# Bài thực hành 5 - Phân tích dữ liệu

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations

In [ ]:
# Tải dữ liệu từ các tệp CSV
customers_df = pd.read_csv('Customers.csv')
order_details_df = pd.read_csv('Order-details.csv')
orders_df = pd.read_csv('Orders.csv')
products_df = pd.read_csv('Products.csv')

# Chuyển đổi OrderDate sang datetime
orders_df['OrderDate'] = pd.to_datetime(orders_df['OrderDate'])

## 4.1. Cho biết những khách hàng ở USA

In [ ]:
khach_hang_usa = customers_df[customers_df['Country'] == 'USA']
print(khach_hang_usa)

## 4.2. Cho biết tên, đơn giá, số lượng tồn kho của những sản phẩm tồn kho hơn 100 sản phẩm

In [ ]:
san_pham_ton_kho_nhieu = products_df[products_df['UnitsInStock'] > 100][['ProductName', 'UnitPrice', 'UnitsInStock']]
print(san_pham_ton_kho_nhieu)

## 4.3. Cho biết những đơn hàng trong quý 2 năm 1997

In [ ]:
don_hang_quy2_1997 = orders_df[(orders_df['OrderDate'].dt.year == 1997) & (orders_df['OrderDate'].dt.quarter == 2)]
print(don_hang_quy2_1997)

## 4.4. Cho biết tên khách hàng (công ty) của đơn hàng có mã 10252

In [ ]:
don_hang_10252 = orders_df[orders_df['OrderID'] == 10252]
customer_id_10252 = don_hang_10252['CustomerID'].iloc[0]
ten_khach_hang = customers_df[customers_df['CustomerID'] == customer_id_10252]['CompanyName'].iloc[0]
print(f'Tên khách hàng của đơn hàng 10252 là: {ten_khach_hang}')

## 4.5. Đếm số mặt hàng trong từng đơn hàng

In [ ]:
so_mat_hang_tung_don = order_details_df.groupby('OrderID')['ProductID'].count().reset_index(name='SoMatHang')
print(so_mat_hang_tung_don)

## 4.6. Tính số đơn hàng và tổng tiền của khách hàng có mã đơn hàng 10252

In [ ]:
customer_id_for_order_10252 = orders_df[orders_df['OrderID'] == 10252]['CustomerID'].iloc[0]
don_hang_cua_khach = orders_df[orders_df['CustomerID'] == customer_id_for_order_10252]
so_don_hang = len(don_hang_cua_khach)
chi_tiet_don_hang_cua_khach = pd.merge(don_hang_cua_khach, order_details_df, on='OrderID')
chi_tiet_don_hang_cua_khach['ThanhTien'] = chi_tiet_don_hang_cua_khach['UnitPrice'] * chi_tiet_don_hang_cua_khach['Quantity'] * (1 - chi_tiet_don_hang_cua_khach['Discount'])
tong_tien = chi_tiet_don_hang_cua_khach['ThanhTien'].sum()
print(f'Khách hàng có mã đơn hàng 10252 là {customer_id_for_order_10252}')
print(f'Số đơn hàng của khách hàng này: {so_don_hang}')
print(f'Tổng tiền của khách hàng này: {tong_tien:.2f}')

## 4.7. Cho biết số lượng đơn hàng và tổng tiền bán từng ngày trong tháng 1 năm 1997

In [ ]:
don_hang_t1_1997 = orders_df[(orders_df['OrderDate'].dt.year == 1997) & (orders_df['OrderDate'].dt.month == 1)]
chi_tiet_t1_1997 = pd.merge(don_hang_t1_1997, order_details_df, on='OrderID')
chi_tiet_t1_1997['ThanhTien'] = chi_tiet_t1_1997['UnitPrice'] * chi_tiet_t1_1997['Quantity'] * (1 - chi_tiet_t1_1997['Discount'])
ket_qua_t1_1997 = chi_tiet_t1_1997.groupby(chi_tiet_t1_1997['OrderDate'].dt.date).agg(SoLuongDonHang=('OrderID', 'nunique'), TongTienBan=('ThanhTien', 'sum')).reset_index()
print(ket_qua_t1_1997)

## 4.8. Tính số lượng, tổng tiền, đơn giá bán bình quân của từng mặt hàng

In [ ]:
order_details_df['ThanhTien'] = order_details_df['UnitPrice'] * order_details_df['Quantity'] * (1 - order_details_df['Discount'])
thong_ke_mat_hang = order_details_df.groupby('ProductID').agg(TongSoLuong=('Quantity', 'sum'), TongTien=('ThanhTien', 'sum')).reset_index()
thong_ke_mat_hang['DonGiaBinhQuan'] = thong_ke_mat_hang['TongTien'] / thong_ke_mat_hang['TongSoLuong']
thong_ke_mat_hang = pd.merge(thong_ke_mat_hang, products_df[['ProductID', 'ProductName']], on='ProductID')
print(thong_ke_mat_hang[['ProductName', 'TongSoLuong', 'TongTien', 'DonGiaBinhQuan']])

## 4.9. Cho biết tên những mặt hàng chưa bán được

In [ ]:
mat_hang_da_ban = order_details_df['ProductID'].unique()
mat_hang_chua_ban = products_df[~products_df['ProductID'].isin(mat_hang_da_ban)]
print(mat_hang_chua_ban[['ProductName']])

## 4.10. Cho biết tên những mặt hàng bán nhiều nhất theo số lượng

In [ ]:
so_luong_ban_tung_mat_hang = order_details_df.groupby('ProductID')['Quantity'].sum().reset_index()
max_quantity = so_luong_ban_tung_mat_hang['Quantity'].max()
mat_hang_ban_nhieu_nhat_id = so_luong_ban_tung_mat_hang[so_luong_ban_tung_mat_hang['Quantity'] == max_quantity]
mat_hang_ban_nhieu_nhat = pd.merge(mat_hang_ban_nhieu_nhat_id, products_df, on='ProductID')
print(mat_hang_ban_nhieu_nhat[['ProductName', 'Quantity']])

## 4.11. Vẽ biểu đồ cột biểu diễn thành tiền bán của 10 mặt hàng bán nhiều nhất trong năm 1997

In [ ]:
don_hang_1997 = orders_df[orders_df['OrderDate'].dt.year == 1997]
chi_tiet_1997 = pd.merge(don_hang_1997, order_details_df, on='OrderID')
chi_tiet_1997['ThanhTien'] = chi_tiet_1997['UnitPrice'] * chi_tiet_1997['Quantity'] * (1 - chi_tiet_1997['Discount'])
top_10_ban_nhieu_nhat = chi_tiet_1997.groupby('ProductID')['ThanhTien'].sum().nlargest(10).reset_index()
top_10_ban_nhieu_nhat_ten = pd.merge(top_10_ban_nhieu_nhat, products_df, on='ProductID')

plt.figure(figsize=(12, 6))
plt.bar(top_10_ban_nhieu_nhat_ten['ProductName'], top_10_ban_nhieu_nhat_ten['ThanhTien'])
plt.xlabel('Tên mặt hàng')
plt.ylabel('Thành tiền bán')
plt.title('Top 10 mặt hàng bán nhiều nhất năm 1997')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4.12. Vẽ biểu đồ cột so sánh số tiền bán được theo tháng của từng năm

In [ ]:
full_details_df = pd.merge(orders_df, order_details_df, on='OrderID')
full_details_df['ThanhTien'] = full_details_df['UnitPrice'] * full_details_df['Quantity'] * (1 - full_details_df['Discount'])
full_details_df['Year'] = full_details_df['OrderDate'].dt.year
full_details_df['Month'] = full_details_df['OrderDate'].dt.month

doanh_thu_thang_nam = full_details_df.groupby(['Year', 'Month'])['ThanhTien'].sum().unstack(level=0)

doanh_thu_thang_nam.plot(kind='bar', figsize=(14, 7), width=0.8)
plt.xlabel('Tháng')
plt.ylabel('Số tiền bán được')
plt.title('So sánh số tiền bán được theo tháng của từng năm')
plt.xticks(rotation=0)
plt.legend(title='Năm')
plt.tight_layout()
plt.show()

## 4.13. Vẽ biểu đồ đường biểu diễn số lượng bán được từng tháng/năm của mặt hàng có nhiều đơn hàng nhất

In [ ]:
so_don_hang_tung_mat = order_details_df.groupby('ProductID')['OrderID'].nunique().reset_index()
max_orders_product_id = so_don_hang_tung_mat.loc[so_don_hang_tung_mat['OrderID'].idxmax()]['ProductID']
ten_mat_hang_nhieu_don = products_df[products_df['ProductID'] == max_orders_product_id]['ProductName'].iloc[0]

chi_tiet_mat_hang_nhieu_don = full_details_df[full_details_df['ProductID'] == max_orders_product_id]
so_luong_ban_thang_nam = chi_tiet_mat_hang_nhieu_don.groupby(['Year', 'Month'])['Quantity'].sum().reset_index()
so_luong_ban_thang_nam['ThangNam'] = so_luong_ban_thang_nam.apply(lambda row: f'{int(row.Month)}-{int(row.Year)}', axis=1)

plt.figure(figsize=(14, 7))
plt.plot(so_luong_ban_thang_nam['ThangNam'], so_luong_ban_thang_nam['Quantity'], marker='o')
plt.xlabel('Tháng/Năm')
plt.ylabel('Số lượng bán được')
plt.title(f'Số lượng bán được của mặt hàng "{ten_mat_hang_nhieu_don}" theo thời gian')
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.tight_layout()
plt.show()

## 4.14. Cho biết những cặp khách hàng mua giống nhau từ 5 mặt hàng trở lên, đó là những mặt hàng nào?

In [ ]:
customer_products = full_details_df.groupby('CustomerID')['ProductID'].apply(set)
customer_pairs = list(combinations(customer_products.index, 2))

for c1, c2 in customer_pairs:
    common_products = customer_products[c1].intersection(customer_products[c2])
    if len(common_products) >= 5:
        company_c1 = customers_df[customers_df['CustomerID'] == c1]['CompanyName'].iloc[0]
        company_c2 = customers_df[customers_df['CustomerID'] == c2]['CompanyName'].iloc[0]
        product_names = products_df[products_df['ProductID'].isin(common_products)]['ProductName'].tolist()
        
        print(f'Cặp khách hàng: {company_c1} và {company_c2}')
        print(f'Số mặt hàng chung: {len(common_products)}')
        print(f'Các mặt hàng chung: {", ".join(product_names)}')
        print('-' * 40)

## 4.15. Cho biết trong những cặp khách hàng (c1, c2) mua giống nhau hơn 5 mặt hàng thì những mặt hàng nào khách hàng c1 mua mà khách hàng c2 chưa mua.

In [ ]:
for c1, c2 in customer_pairs:
    common_products = customer_products[c1].intersection(customer_products[c2])
    if len(common_products) > 5:
        c1_only_products = customer_products[c1].difference(customer_products[c2])
        c2_only_products = customer_products[c2].difference(customer_products[c1])
        
        company_c1 = customers_df[customers_df['CustomerID'] == c1]['CompanyName'].iloc[0]
        company_c2 = customers_df[customers_df['CustomerID'] == c2]['CompanyName'].iloc[0]
        
        if c1_only_products:
            product_names_c1_only = products_df[products_df['ProductID'].isin(c1_only_products)]['ProductName'].tolist()
            print(f'Trong cặp ({company_c1}, {company_c2}):')
            print(f'  {company_c1} mua những mặt hàng sau mà {company_c2} không mua:')
            print(f'    { ", ".join(product_names_c1_only) }')
        
        if c2_only_products:
            product_names_c2_only = products_df[products_df['ProductID'].isin(c2_only_products)]['ProductName'].tolist()
            print(f'Trong cặp ({company_c1}, {company_c2}):')
            print(f'  {company_c2} mua những mặt hàng sau mà {company_c1} không mua:')
            print(f'    { ", ".join(product_names_c2_only) }')
        
        if c1_only_products or c2_only_products:
             print('-' * 40)